In [1]:
import os
import glob
import pandas as pd
import argparse
import xml.etree.ElementTree as ET

In [2]:
def xml_to_csv(input_path=None, output_csv_path=None):
    # if input variables are not defined, then put them in current working directory and input_path separately
    if(input_path==None):
        input_path = os.getcwd()
    if(output_csv_path==None):
        output_csv_path = os.getcwd()
    
    xml_list = []
    for xml_file in os.listdir(input_path):
        xml_file_path = os.path.join(input_path, xml_file)
        
        # parse the xml file
        tree = ET.parse(xml_file_path) 
        root = tree.getroot()

        # root.findall('object'): find the <object> tag which can lead to the number of labeled items
        # better open the .xml file to check the tags and infos inside
        for member in root.findall('object'):
            value = (root.find('filename').text,      # info of image file name
                     int(root.find('size')[0].text),  # width of the image
                     int(root.find('size')[1].text),  # height of the image
                                                      # depth of the image can be found by -- int(root.find('size')[2].text)
                     member[0].text,                  # the first tag under object tag which is the label
                     int(member[1][0].text),          # the label box xmin
                     int(member[1][1].text),          # the label box ymin
                     int(member[1][2].text),          # the label box xmax
                     int(member[1][3].text),          # the label box ymax
                     )
            xml_list.append(value)

    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    
    # write the data into csv
    csv_file = os.path.join(output_csv_path, "label.csv")
    xml_df.to_csv(csv_file, index=False)

    print('Successfully converted xml to csv.')

In [3]:
# get image path
project04_root = os.path.dirname(os.getcwd())
img_path = os.path.join(project04_root, "images")
print("Path for images: {}".format(img_path))
print("------")

# get image path of train and test separately
img_path_test, img_path_train = [os.path.join(img_path, train_test) for train_test in os.listdir(img_path)]
print("Path for train images: {}".format(img_path_train))
print("Path for test images: {}".format(img_path_test))
print("------")

# get path for annotation of train dataset
train_annotation_path = os.path.join(img_path_train, "train_annotations")
print("Path for train annotation: {}".format(train_annotation_path))

# csv output path
csv_path = os.getcwd()
print("Path for saving csv: {}".format(csv_path))

Path for images: D:\DeepLearning\Project04_ContainerDetection\images
------
Path for train images: D:\DeepLearning\Project04_ContainerDetection\images\train_cdc
Path for test images: D:\DeepLearning\Project04_ContainerDetection\images\test_cdc
------
Path for train annotation: D:\DeepLearning\Project04_ContainerDetection\images\train_cdc\train_annotations
Path for saving csv: D:\DeepLearning\Project04_ContainerDetection\workspace


In [4]:
xml_to_csv(train_annotation_path, csv_path)

Successfully converted xml to csv.


In [ ]:
"""
Usage:

# Create train data:
python generate_tfrecord.py --label=<LABEL> --csv_input=<PATH_TO_ANNOTATIONS_FOLDER>/train_labels.csv  --output_path=<PATH_TO_ANNOTATIONS_FOLDER>/train.record

# Create test data:
python generate_tfrecord.py --label=<LABEL> --csv_input=<PATH_TO_ANNOTATIONS_FOLDER>/test_labels.csv  --output_path=<PATH_TO_ANNOTATIONS_FOLDER>/test.record
"""

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('label', '', 'Name of class label')
# if your image has more labels input them as
# flags.DEFINE_string('label0', '', 'Name of class[0] label')
# flags.DEFINE_string('label1', '', 'Name of class[1] label')
# and so on.
flags.DEFINE_string('img_path', '', 'Path to images')
FLAGS = flags.FLAGS


# TO-DO replace this with label map
# for multiple labels add more else if statements
def class_text_to_int(row_label):
    if row_label == FLAGS.label:  # 'ship':
        return 1
    # comment upper if statement and uncomment these statements for multiple labelling
    # if row_label == FLAGS.label0:
    #   return 1
    # elif row_label == FLAGS.label1:
    #   return 0
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(os.getcwd(), FLAGS.img_path)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.app.run()